In [1]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision
from torchvision import models
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter


In [2]:
class HotdogData(Dataset):
    def __init__(self, img_path, transforms=None):
        # 初始化，读取数据集
        self.transforms = transforms
        self.img_path = img_path
        self.pos_dir = img_path + '/height-limit'
        self.neg_dir = img_path + '/not-height-limit'
        self.pos_dirs = os.listdir(self.pos_dir)
        self.neg_dirs = os.listdir(self.neg_dir)
        self.pos_num = len(self.pos_dirs)
        self.neg_num = len(self.neg_dirs)
        
        
    def __len__(self):
        return self.pos_num + self.neg_num
    
    def __getitem__(self, index):
        if index < self.pos_num: # 获取正样本
            label = 1
#             path=self.pos_dir + '/' + str(index) + '.png'
#             print(path)
            img = Image.open(self.pos_dir + '/' + self.pos_dirs[index])
        else: # 获取负样本
            label = 0
            img = Image.open(self.neg_dir + '/' + self.neg_dirs[index-self.pos_num])
            
        if self.transforms:
            img = self.transforms(img)
            
        return img, label

In [3]:
train_transform = transforms.Compose([
                    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)), # 将图像随意裁剪，宽高均为224
                    transforms.RandomHorizontalFlip(), # 以0.5的概率左右翻转图像
                    transforms.RandomVerticalFlip(),
#                     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0),
                    transforms.RandomRotation(degrees=5, expand=False, fill=None),
                    transforms.ToTensor(), # 将PIL图像转为Tensor，并且进行归一化
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化
                ])
test_transform = transforms.Compose([
                    transforms.Resize(256), 
                    transforms.CenterCrop(224),
                    transforms.ToTensor(), # 将PIL图像转为Tensor，并且进行归一化
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化
                ])

train_data = HotdogData('/openbayes/input/input0/classification-train', transforms=train_transform)
trainloader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = HotdogData('/openbayes/input/input0/classification-test', transforms=test_transform)
testloader = DataLoader(test_data, batch_size=64, shuffle=True)


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = models.resnet18(pretrained=True, progress=True)
net = net.to(device)

In [5]:
dummy_input = torch.rand(13, 3, 224, 224).cuda()
with SummaryWriter('runs/exp-1') as w:
    w.add_graph(net, (dummy_input,))

In [6]:


# 全连接层的输入通道in_channels个数
num_fc_in = net.fc.in_features

# 改变全连接层，2分类问题，out_features = 2
net.fc = nn.Linear(num_fc_in, 2).cuda()

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
lr = 0.001 / 10
fc_params = list(map(id, net.fc.parameters())) # 取得全连接层的参数内存地址的列表
base_params = filter(lambda p: id(p) not in fc_params, net.parameters()) # 取得其他层参数的列表
optimizer = optim.Adam([
            {'params': base_params},
            {'params': net.fc.parameters(), 'lr': lr * 10}],
            lr=lr, betas=(0.9, 0.999))

In [9]:
epoch_num = 2
evaluate_batch_num = 5

for epoch in range(epoch_num):  # loop over the dataset multiple times
    running_loss = 0.0
    epoch_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs=inputs.cuda()
        labels=labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).cuda()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_loss += loss.item()
        if i % evaluate_batch_num == evaluate_batch_num - 1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch, i + 1, running_loss / evaluate_batch_num))
            
            with SummaryWriter('runs/exp-1') as w:
                w.add_scalar('TrainLoss/epoch' + str(epoch), running_loss / evaluate_batch_num, i // evaluate_batch_num)             
            running_loss = 0.0
            
    with SummaryWriter('runs/exp-1') as w:
        w.add_scalar('TrainLoss/all', epoch_loss / len(trainloader), epoch)
        epoch_loss = 0.0


torch.save(net.state_dict(),"model.pt")     
    
print('Finished Training')


[0,     5] loss: 0.410
[0,    10] loss: 0.038
[0,    15] loss: 0.078
[0,    20] loss: 0.063
[0,    25] loss: 0.054
[0,    30] loss: 0.070
[0,    35] loss: 0.037
[0,    40] loss: 0.040
[0,    45] loss: 0.011
[0,    50] loss: 0.005
[0,    55] loss: 0.018
[0,    60] loss: 0.019
[0,    65] loss: 0.006
[0,    70] loss: 0.005
[0,    75] loss: 0.005
[0,    80] loss: 0.006
[0,    85] loss: 0.016
[0,    90] loss: 0.004
[0,    95] loss: 0.003
[0,   100] loss: 0.002
[0,   105] loss: 0.004
[0,   110] loss: 0.003
[0,   115] loss: 0.001
[0,   120] loss: 0.009
[0,   125] loss: 0.009
[0,   130] loss: 0.013
[0,   135] loss: 0.003
[0,   140] loss: 0.003
[0,   145] loss: 0.019
[0,   150] loss: 0.003
[0,   155] loss: 0.024
[0,   160] loss: 0.004
[0,   165] loss: 0.022
[0,   170] loss: 0.005
[0,   175] loss: 0.003
[0,   180] loss: 0.005
[0,   185] loss: 0.009
[0,   190] loss: 0.002
[0,   195] loss: 0.001
[0,   200] loss: 0.002
[0,   205] loss: 0.002
[0,   210] loss: 0.001
[0,   215] loss: 0.014
[0,   220] 

In [10]:
correct = 0
total = 0
net=net.cpu()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print((labels,predicted))
        correct += (predicted == labels).sum().item()

        print((correct,total))
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

(tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(63, 64)
(tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]), tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(122, 128)
(tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0,

In [11]:
# 载入模型并预测
m_state_dict = torch.load('model.pt')
new_model= models.resnet18(pretrained=False, progress=True)
num_fc_in = new_model.fc.in_features

new_model.fc = nn.Linear(num_fc_in, 2)

new_model.load_state_dict(m_state_dict)

new_model.eval()

im_path="/openbayes/input/input0/train/no/163.png"
img=Image.open(im_path)
test_transform = transforms.Compose([
                    transforms.Resize(256), 
                    transforms.CenterCrop(224),
                    transforms.ToTensor(), # 将PIL图像转为Tensor，并且进行归一化
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化
                ])
img_=test_transform(img).unsqueeze(0)

outputs = new_model(img_)
print(outputs)
_, predicted = torch.max(outputs.data, 1)
print(torch.max(outputs.data, 1))
print(predicted)

FileNotFoundError: [Errno 2] No such file or directory: '/openbayes/input/input0/train/no/163.png'